<a href="https://colab.research.google.com/github/Nubiancodingdelight/ACS-Project-Repository-/blob/main/Generate_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebooks generates matrices as numpy arrays.
Created By: Lorrayya Williams
Updated On: 4/13/2025

#Set up


In [1]:
#mounts google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#sets path
%cd /content/drive/MyDrive/ACS_Research/VISDB_Data/

/content/drive/MyDrive/ACS_Research/VISDB_Data


INSTALL Packages

In [3]:
!pip install pysam

#Imports

In [4]:
#imports
import numpy as np
import subprocess
import shlex
import pysam
import re
import re
import math
import statistics as stat


# Functions




## Generate Cigar

In [5]:
def generate_cigar(ref, seq): # inputs the reference sequence and the sample sequence
    cigar = []
    count = 0
    op = ''

    #loops through reference and sample sequence which are already aligned
    for r, s in zip(ref, seq):
        # Determine operation
        if r == s and r != '-':
            current_op = 'M'
        elif r != s and r != '-' and s != '-':
            current_op = 'X'
        elif r == '-' and s != '-':
            current_op = 'I'
        elif s == '-' and r != '-':
            current_op = 'D'
        else:
            continue  # skip if both are gaps

        # Group by same operation
        if current_op == op:
            count += 1
        else:
            if op:
                cigar.append(f"{count}{op}")
            op = current_op
            count = 1

    if op:
        cigar.append(f"{count}{op}")

    return ''.join(cigar)  #returns a string that represents the cigar value

##Reverse Complete

In [6]:
def reverse_complement(seq):
  #generates the opposing strand from the reference
    complement = str.maketrans('ACGTNacgtn', 'TGCANtgcan')
    return seq.translate(complement)

## Create Bam from Sequence

In [7]:
import pysam
from pysam import AlignmentHeader, AlignedSegment
def create_bam_from_sequence(output_bam_path, dna_sequence,ref_seq, reference_name, start_pos):
  """
  Creates a BAM file from a DNA sequence and its coordinates.

  Args:
      output_bam_path (str): Path to save the BAM file (e.g., "output.bam").
      dna_sequence (str): DNA sequence (e.g., "ATCGATCG").
      reference_name (str): Reference sequence name (e.g., "chr1").
      start_pos (int): 1-based start position on the reference.
  """
  # 1. Create a BAM header
  header = AlignmentHeader.from_references(
      [reference_name],  # List of reference names
      [1]  # Lengths of references
      )

  # 2. Open a BAM file for writing
  with pysam.AlignmentFile(output_bam_path, "wb", header=header) as bam_file:
    # 3. Create an aligned segment (read)
    read = AlignedSegment(header)
    read.query_name = "read1"  # Read ID
    read.query_sequence = dna_sequence  # DNA sequence
    read.flag = 0  # No flags set (0 means mapped)
    read.reference_id = 0  # Index of reference in header (0 = first one)
    read.reference_start = start_pos - 1  # 0-based position
    read.mapping_quality = 60  # High mapping quality
    read.cigarstring = generate_cigar(dna_sequence,ref_seq)  # CIGAR string (exact match)

    # 4. Write the read to the BAM file
    bam_file.write(read)

     # Reverse strand read
    read_rev = AlignedSegment(header)
    read_rev.query_name = "read2"
    rev_seq = reverse_complement(ref_seq[0:2000])
    rev_ref = reverse_complement(ref_seq[0:2000])
    read_rev.query_sequence = rev_seq
    read_rev.flag = 16  # Reverse strand flag
    read_rev.reference_id = 0
    read_rev.reference_start = start_pos - 1
    read_rev.mapping_quality = 60
    read_rev.cigarstring = generate_cigar(rev_seq,rev_ref)
    bam_file.write(read_rev)

    return read, read_rev


##Insertion Span

In [8]:
def insertion_span(vir_start, vir_end, ref_start, ref_end):
  #generates the span that is missing in the reference genome due to viral insertion
  import math
  diff = abs(vir_start- vir_end)
  dir =  [0,1] if vir_start < vir_end else [1,0]
  return min(ref_start, ref_end), min(ref_start, ref_end) -diff, dir

#Matrix Generator 1 -- DeepHBV


In [9]:
def extract_virus(samfile,start, end):
  for line in samfile:
    line_list = str(line).split("\t")
    CIGAR = line_list[5]
    SEQ = line_list[9]
    POS = line_list[3]
    SEQ = line_list[9]
    if int(POS) in range(start,end):
      POS_SEQ.append(SEQ)
    else:
      NEG_SEQ.append(SEQ)
  return POS_SEQ, NEG_SEQ


In [10]:
import numpy as np
import os
from scipy.io import loadmat

def matrix_generator_uno(seq, isvirus=True):  # One Hot Encoding
    #seq_list = list(seq)
    #print(seq)
    tensor = np.zeros((1, len(seq), 4))
    if isvirus:
      label = np.ones(1)
    else:
      label = np.zeros(1)
    for i in range(len(seq)):
      if seq[i].upper() == 'A':
          tensor[0][i] = [1, 0, 0, 0]
      if seq[i].upper() == 'T':
          tensor[0][i] = [0, 1, 0, 0]
      if seq[i].upper() == 'C':
          tensor[0][i] = [0, 0, 1, 0]
      if seq[i].upper() == 'G':
          tensor[0][i] = [0, 0, 0, 1]
    return tensor, label

# Matrix Generator 2 -- Novel Matrix

In [11]:
#matrix row generator from CIGAR, SEQuence, Position of Insert
def matrix_row(CIG, SEQ, POS, start, end, isVirus = True):

  #intialize values
  matrix_list= []
  label_list =[]
  bases = ['A','T','C','G']
  seq_list= list(SEQ)
  cigar_list = re.split(r'(\d+)', CIG)[1::]
  tensor= np.zeros((1,2000, 4))

  #create label
  if int(POS) in range(start,end) or isVirus:
    label_list.append(1)
  else:
    label_list.append(0)

  #if the Cigar length is short
  counter =0
  if len(cigar_list) ==2:
    #if it is a matched a sequence of 1's based on the base are added
    if cigar_list[1] == 'M':
      i= 0
      for j in range(int(len(cigar_list[0]))):
        num_list = [0,0,0,0]
        try:
          num_list[bases.index(seq_list[j])] = 1
          tensor[i][j]= num_list
        except ValueError:
          tensor[i][j]= num_list

  #for CIGAR lists longer than 2
  else:
    length = 0
    i = 0
    for time in range(int(len(cigar_list)/2)):
      if cigar_list[(2*time)+1] == 'M':
        for j in range(length,length + int(cigar_list[(2*time)])):
          num_list = [0,0,0,0]
          try:
            num_list[bases.index(seq_list[j])] = 1
            counter += length
            tensor[0][j]= num_list
          except ValueError:
            tensor[0][j]= num_list
        length += int(cigar_list[(2*time)])
  return tensor, label_list #return tensor and label


In [12]:
def matrix_generator_dos (read1,read2, start, end, isVirus=True):

  #intializes values
  final_matrix = []
  label_list = []
  reads =[read1, read2]

  #loops through reads
  for read in reads:

    #extracts values from each read
    CIGAR = read.cigarstring
    SEQ = read.query_sequence
    POS = read.reference_start
    final_matrix_temp, label_list_temp = matrix_row(CIGAR, SEQ, POS, start,end,isVirus)

    if len(final_matrix) == 0:
      final_matrix = final_matrix_temp
    else:
      final_matrix = np.concatenate([final_matrix, final_matrix_temp])
    label_list += label_list_temp

  #save lists as numpy arrays
  Data = np.asarray(final_matrix)
  Label = np.asarray(label_list)
  return Data, Label

# Matrix Generator 3 -- 3D Matrix

> Add blockquote



In [13]:
def generate_mapping_list(read1,read2, int_start, int_end, direction):
  #intializes values
  reads = [read1, read2]
  mapping_list = []

  #loops through reads
  for ind in range(len(reads)):

    #extract values from read
    CIGAR = re.split(r'(\d+)', reads[ind].cigarstring)[1::]
    SEQ =  list(reads[ind].query_sequence)
    POS = int(reads[ind].reference_start)
    DIR = direction[ind]
    COUNT= 0

    if len(CIGAR) ==2:
      if CIGAR[1] == 'M':
        mat = 1
        for i in range(len(SEQ)):
          mapping_list.append([SEQ[i].upper(), mat, int(POS) ,DIR, COUNT])
          POS +=1
      else:
        mat=0
        for i in range(len(SEQ)):
          mapping_list.append([SEQ[i].upper(), mat, int(POS),DIR, COUNT])
          POS +=1

    else:
      length = 0
      i = 0
      for time in range(int(len(CIGAR)/2)):
        if CIGAR[(2*time)+1] == 'M':
          mat =1
          for j in range(length,length + int(CIGAR[(2*time)])):
            mapping_list.append([SEQ[j].upper(), mat, int(POS) ,DIR, COUNT])
            POS +=1
          length += int(CIGAR[(2*time)])
        else:
          mat =0
          for j in range(length,length + int(CIGAR[(2*time)])):
            mapping_list.append([SEQ[j].upper(), mat, int(POS) ,DIR, COUNT])
            POS +=1
            i += 1
          length += int(CIGAR[(2*time)])
        COUNT += 1


  #make list into dataframe
  df = pd.DataFrame(mapping_list, columns = ['Base', 'Match', 'Position', 'Direction', 'Row_count'])
  df = df.sort_values(by=['Position'])
  df = df.reset_index(drop=True)
  #extract three_five
  three_five = df[df['Direction'] ==0]
  three_five = three_five.reset_index(drop=True)

  #extract five_three
  five_three = df[df['Direction'] ==1]
  five_three = five_three.reset_index(drop=True)
  return df, three_five, five_three

In [29]:
def matrix_generator_tres(read1, read2, int_start, int_end, direction, isVirus =True):

  df, three_five, five_three = generate_mapping_list(read1,read2, int_start, int_end, direction)

  #intialize tensor
  tensor = np.zeros((1,2000, 4))
  label_list= []

  #create label
  if isVirus:
    label_list.append(1)
  else:
    label_list.append(0)
  label = np.asarray(label_list)

  #initialize bases as well as base dict
  bases = ['A','T','C','G']
  match_dict ={'A':'T', 'T':'A', 'C':'G', 'G':'C'}

  for row in range(len(three_five)):
    try:
      if three_five['Base'].iloc[row] in bases and three_five['Match'].iloc[row] == 1:
        tensor[0][row][bases.index(three_five['Base'].iloc[row])] = 1
        if str(five_three['Base'].iloc[row]) == match_dict[str(three_five['Base'].iloc[row])]:
          tensor[0][row][bases.index(three_five['Base'].iloc[row])] =3
      elif str(five_three['Base'].iloc[row]) == match_dict[str(three_five['Base'].iloc[row])]:
          tensor[0][row][bases.index(three_five['Base'].iloc[row])]=2
    except:
      pass
  return tensor, label #returns a tensor and labels

#TESTING

In [15]:
#load hbv data
import pandas as pd
hbv_data = pd.read_csv('Spliced_Data_HBV.csv')
hbv_data = hbv_data.iloc[1:]
hbv_data = hbv_data[hbv_data['spliced_seq'].str.len() == 2000]

In [16]:
#Testing
matrix, label =matrix_generator_uno(str(hbv_data['spliced_seq'][1]), True)
print(matrix.shape)
print(label.shape)

(1, 2000, 4)
(1,)


In [18]:
#Testing Code
read1, read2 = create_bam_from_sequence('/content/drive/MyDrive/ACS_Research/VISDB_Data/Aligned_Sequences/HBV/HBV2.bam',hbv_data['spliced_seq'][2], hbv_data['human_ref_sequence'][2], 'hg19', hbv_data['begin_breakpoint'][2])
start,end,direction = insertion_span(hbv_data['begin_ref'][2], hbv_data['stop_ref'][2], hbv_data['begin_breakpoint'][2], hbv_data['stop_breakpoint'][2])
Data2,Label2 =matrix_generator_dos (read1, read2, start, end)
print(Data2.shape)
print(Label2.shape)

(2, 2000, 4)
(2,)


In [19]:
read1, read2 = create_bam_from_sequence('/content/drive/MyDrive/ACS_Research/VISDB_Data/Aligned_Sequences/HBV/HBV2.bam',hbv_data['spliced_seq'][2], hbv_data['human_ref_sequence'][2], 'hg19', hbv_data['begin_breakpoint'][2])
start,end,direction = insertion_span(hbv_data['begin_ref'][2], hbv_data['stop_ref'][2], hbv_data['begin_breakpoint'][2], hbv_data['stop_breakpoint'][2])
Data3,Label3 =matrix_generator_tres (read1, read2, start, end, direction, isVirus=False)

# HBV

In [20]:
#load hbv data
import pandas as pd
hbv_data = pd.read_csv('Spliced_Data_HBV.csv')
hbv_data = hbv_data[hbv_data['spliced_seq'].str.len() == 2000] #clean data
hbv_data.head()

,Unnamed: 0,virus,virus_ref,begin_ref,stop_ref,human_ref,begin_breakpoint,stop_breakpoint,spliced_seq,viral_seq,human_ref_sequence,human_seq_upstream,human_seq_downstream
0,1,HBV,X70185.1,97732036,97732056,GRCh37/hg19,1246,1390,TTACTGCTGATCACCAGGTTTTTTAGCTAATAGGGACATCTAGAAA...,TTTGTGGCTCCTCTGCCGATCCATACTGCGGAACTCCTAGCCGCTT...,TTACTGCTGATCACCAGGTTTTTTAGCTAATAGGGACATCTAGAAA...,TTACTGCTGATCACCAGGTTTTTTAGCTAATAGGGACATCTAGAAA...,gtaatttgtaatgaacagatattcatttCtttttttattattatta...
1,2,HBV,X70185.1,194161891,194161910,GRCh37/hg19,1603,1664,CATTTTAGAGAAAATTAAACTGTATTCTGCTTTAAAATATATCCAG...,GTTGCATGGAGACCACCGTGAACGCCCATCAGATCCTGCCCAAGGT...,CATTTTAGAGAAAATTAAACTGTATTCTGCTTTAAAATATATCCAG...,CATTTTAGAGAAAATTAAACTGTATTCTGCTTTAAAATATATCCAG...,AGTGTTTGCTTTTTAAAATGTTAAGATTAAATCATATTGTTACttt...
2,3,HBV,X70185.1,171450967,171450816,GRCh37/hg19,1623,1717,ctagtaagccagtgtttcctatgttgggtaaacacatttgcagtgg...,AACGCCCATCAGATCCTGCCCAAGGTCTTACATAAGAGGACTCTTG...,cttgaacatgaatgcagccttggccttccactgttttgtttccAGC...,ctagtaagccagtgtttcctatgttgggtaaacacatttgcagtgg...,cttgaacatgaatgcagccttggccttccactgttttgtttccAGC...
3,4,HBV,X70185.1,171450967,171450816,GRCh37/hg19,1638,1717,ctagtaagccagtgtttcctatgttgggtaaacacatttgcagtgg...,CTGCCCAAGGTCTTACATAAGAGGACTCTTGGACTCTCAGCAATGT...,ttatggttctgatggggccgtcaggcatattacactaaaccctggc...,ctagtaagccagtgtttcctatgttgggtaaacacatttgcagtgg...,ttatggttctgatggggccgtcaggcatattacactaaaccctggc...
4,5,HBV,X70185.1,171450967,171450779,GRCh37/hg19,1645,1717,ctagtaagccagtgtttcctatgttgggtaaacacatttgcagtgg...,AGGTCTTACATAAGAGGACTCTTGGACTCTCAGCAATGTCAACGAC...,GGCTCGCTGATATTGTACATTCTGATATTCATAAAGACACTGTCAC...,ctagtaagccagtgtttcctatgttgggtaaacacatttgcagtgg...,GGCTCGCTGATATTGTACATTCTGATATTCATAAAGACACTGTCAC...


In [21]:
for index, row in hbv_data.iterrows():
  try:

    #Generate Bam
    bam_file_path = '/content/drive/MyDrive/ACS_Research/VISDB_Data/Aligned_Sequences/'  +str(row['virus'])+str(index)+'.bam' # Changed to .bam
    read1, read2 = create_bam_from_sequence(bam_file_path,row['spliced_seq'], row['human_ref_sequence'], 'hg19', row['begin_breakpoint'])

    start,end,direction = insertion_span(row['begin_ref'], row['stop_ref'], row['begin_breakpoint'], row['stop_breakpoint'])

    #generate matrix 1
    Data1, Label1=matrix_generator_uno(row['spliced_seq'])

    #save matrix 1 output
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HBV/Experiment_1/'+str(row['virus'])+str(index)+'_Matrix_data.npy', Data1)
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HBV/Experiment_1/'+str(row['virus'])+str(index)+'_Label_data.npy', Label1)

    #generate matrix 2
    Data2,Label2 =matrix_generator_dos (read1, read2, start, end)

    #save generated data
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HBV/Experiment_2/'+str(row['virus'])+str(index)+'_Matrix_data.npy', Data2)
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HBV/Experiment_2/'+str(row['virus'])+str(index)+'_Label_data.npy', Label2)


    #generate matrix 3
    Data3,Label3 =matrix_generator_tres(read1,read2, start, end, direction)

    #save matrix
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HBV/Experiment_3/'+str(row['virus'])+str(index)+'_Matrix_data.npy', Data3)
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HBV/Experiment_3/'+str(row['virus'])+str(index)+'_Label_data.npy', Label3)

  except:
    print(row['viral_seq'])
    pass

TTCCATGGCTGCTAGGCTGTACTGCCAACTGGATCCTTCGCGGGACGTCCTTTG
ATGTCAACGACCGACCTTGAGGCATACTTCAAAGACTGTTTGTTTAAAGACTGGGAGGAGTTGGGGGAGGAGATTAGGTTAAAGGTCTTT
ACAGCATGGGAGGTTGGTCTTCCAAACCTCGACAAGGCA
CCTGCTGCTATGCCTCATCTTCTTGTTGGTTCTTCTGGACTACCAAGGTATG
TCTGCCTAATCATCTCATGTTCATGTCCTACTGTTCAAGCCTCCAAGCTGTGCCTTGGGTGGCTTTGGGGCATGGACATTGACCCGTATAAA
GACTCATAAGGTGGGAAACTTTACTGGGCTTTATTCTTCTACTGTACCTGTCTTTAATCCTGATTGGAAAACTCCCTCCTTTCCTCACATTCA
TCAGGCAAGCTATTCTGTGTTGGGGTGAGTTGATGAATCTGGCCACCTGGGTGGGAAGTAATTTGGAAGACCCAGCATCCAGGGAATTAGT
CTACTCCCATCTCTCCACCTCTAAGAGACAG
TTCCAAACCTCGACAAGGCATGGGGACGAATCTTTCTGTTCCCAATCCTCTGGGATTCTTTCCCGATCACCAGTTGGACCCTGCGTTCGGAGCCAACTCAAACAATCCAGATTGG
CATGGAGACCACCGTGAACGCCCACCAGGTCTTGCCCAAGGTCTTACACAAGAGGACTCTTGGACTCTCAGCAATGTCAACGACCGACCTTGAGGCATACTTCAAAGACTGTTT
GTACCTGTCTTTAATCCTGATTGGAAAACTCCCTCCTTTCCTCACATTCATTTACAGGAGGACATTATTAATAGATGTCAACAATATGTGGGCCCTCTGACAGTTA
CTTTCCCCCACTGTTTGGCTTTCAGCTATATGGATGATGTGGTATTGGGGGCCAAGTCTGTACAACATCTTGAGTCCCTTTTTACCTCTA
CCTCCTGCCTCCACCAATCGG

# HPV

In [22]:
#load hbv data
import pandas as pd
hpv_data = pd.read_csv('Spliced_Data_HPV.csv')
hpv_data = hpv_data[hpv_data['spliced_seq'].str.len() == 2000]
hpv_data.head()

,Unnamed: 0,virus,virus_ref,begin_ref,stop_ref,human_ref,begin_breakpoint,stop_breakpoint,spliced_seq,viral_seq,human_ref_sequence,human_seq_upstream,human_seq_downstream
0,0,HPV,AF125673.1,28595279,28595577,GRCh37/hg19,2827,3062,TACACATTCACTTGTATTCTATTTCCACCTCACCTCCAATGCCAGC...,AGACCTACGTGACCATATAGACTATTGGAAACACATGCGCCTAGAA...,TACACATTCACTTGTATTCTATTTCCACCTCACCTCCAATGCCAGC...,TACACATTCACTTGTATTCTATTTCCACCTCACCTCCAATGCCAGC...,gattgattaaggtcagagtcatcaaacgatgctaaggctggtgggc...
1,1,HPV,AF125673.1,28595311,28595730,GRCh37/hg19,2505,3062,atgtaaatgatgaattaacgggtgcagcacaccaacatggcacatg...,GGATGTAAAGCATAGACCATTGGTACAACTAAAATGCCCTCCATTA...,atgtaaatgatgaattaacgggtgcagcacaccaacatggcacatg...,atgtaaatgatgaattaacgggtgcagcacaccaacatggcacatg...,ccaacctgacctggtgtcaaggttaacatcactaatggtgggacag...
2,2,HPV,AF125673.1,28595585,28595729,GRCh37/hg19,2173,2575,tggtgattgattGGTGATTGGAAGCAAATTGTTATGTTTTTAAGGT...,GGTGATTGGAAGCAAATTGTTATGTTTTTAAGGTATCAAGGTGTAG...,tggtgattgattaaggtcagagtcatcaaacgatgctaaggctggt...,tggtgattgatt,accaacctgacctggtgtcaaggttaacatcactaatggtgggaca...
3,3,HPV,AF125673.1,28595716,28596131,GRCh37/hg19,3151,3678,catgtatacatatgtaacaaatctgcacgttgtgcacgtgtaccct...,AAACTGGACACATATATATATTTGTGAAGAAGCATCAGTAACTGTG...,catgtatacatatgtaacaaatctgcacgttgtgcacgtgtaccct...,catgtatacatatgtaacaaatctgcacgttgtgcacgtgtaccct...,cctaatttggaggatttaatgatagtaacaaggagaagatgttgtg...
4,4,HPV,AF125673.1,28595741,28595968,GRCh37/hg19,3693,4038,agggatattgatgttcaactcaggcacaagaagaggatgccaggat...,CTGCAGTGTCGTCTACATGGCATTGGACAGGACATAATGTAAAACA...,agggatattgatgttcaactcaggcacaagaagaggatgccaggat...,agggatattgatgttcaactcaggcacaagaagaggatgccaggat...,tgtttcctattcaaggaaactaat


In [23]:
for index, row in hpv_data.iterrows():
  try:
    bam_file_path = '/content/drive/MyDrive/ACS_Research/VISDB_Data/Aligned_Sequences/'  +str(row['virus'])+str(index)+'.bam' # Changed to .bam
    read1, read2 = create_bam_from_sequence(bam_file_path,row['spliced_seq'], row['human_ref_sequence'], 'hg19', row['begin_breakpoint'])

    start,end,direction = insertion_span(row['begin_ref'], row['stop_ref'], row['begin_breakpoint'], row['stop_breakpoint'])

    #generate matrix 1
    Data1, Label1=matrix_generator_uno(row['spliced_seq'])

    #save matrix 1 output
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HPV/Experiment_1/'+str(row['virus'])+str(index)+'_Matrix_data.npy', Data1)
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HPV/Experiment_1/'+str(row['virus'])+str(index)+'_Label_data.npy', Label1)

    #generate matrix 2
    Data2,Label2 =matrix_generator_dos (read1, read2, start, end)

    #save generated data
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HPV/Experiment_2/'+str(row['virus'])+str(index)+'_Matrix_data.npy', Data2)
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HPV/Experiment_2/'+str(row['virus'])+str(index)+'_Label_data.npy', Label2)


    #generate matrix 3
    Data3,Label3 =matrix_generator_tres(read1,read2, start, end, direction)

    #save matrix
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HPV/Experiment_3/'+str(row['virus'])+str(index)+'_Matrix_data.npy', Data3)
    np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/HPV/Experiment_3/'+str(row['virus'])+str(index)+'_Label_data.npy', Label3)

  except:
    print(row['viral_seq'])
    pass

TTGGCCAACCACTCCGCCGCGACCCATACCAAAGCCGTCGCCTTGGGCACCGAAGAAACACAGACGACTATCCAGCGACCAAGATCAGAGCCAGACACCGGAAACCCCTGCCACACCACTAAGTTGTTGCACAGAGACTCAGTGGACAGTGCTCCAATCCTCACTGCATTTAACAGCTCACACAAAGGACGGATTAACTGTAATAGTAACACTACACCCATAGTACATTTAAAAGGTGATGCTAATACTTTAAAATGTTTAAGATATAGATTTAAAAAGCATTGTACATTGTATACTGCAGTGTCGTCTACATGGCATTGGACAGGACAT
TGAGTTTCCATTTGACGAAAACGGAAATCCAGTGTATGAGCTTAATGATAAGAACTGGAAATCCTTTTTCTCAAGGACGTGGTCCAGATTAAGTTTGCACGAGGACGAGGACAAGGAAAACGATGGAGACTCTTTGCCAACGTTTAAATGTGTGTCAGGACAAAATACTAACACATTATG
ATCCATATGCTGTATGTGATAAATGTTTAAAGTTTTATTCTAAAATTAGTGAGTATAGACATTATTGTTATAGTGTGTATGGAACAACATTAGAACAGCAATACAACAAACCGTTGTGTGATTTGTTAATTAGGTGTATTAACTGTCAAAAGCCACTGTGTCCTGAAGAAAAGCAAAGACATCTGGACAAAAAGCAAAGATTCCATAATATAAGGGGTCGGTGGACCGGTCGATGTATGTCTTGTTGCAGATCATCAAGAACACGTAGAGAAACCCAGCTGTAATCATGCATGGAGATACACCTACATTGCATGAATATATGTTAGATTTGCAACCAGAGACAACTGATCTCTACTGTTATGAGCAATTAAATGACAGCTCAGAGGAGGAGGATGAAATAGATGGTCCAGCTGGACAAGCAGAACCGGACAGAGCCCATTACAATATTGTAACCTTTTGTTGCAAGTGTGACTCTACGCTTCGGTTGT

# Reference Data


In [24]:
#load hbv data
import pandas as pd
ref_data = pd.read_csv('Spliced_Data_Reference.csv')
ref_data = ref_data[ref_data['human_ref_sequence'].str.len() == 2000]
ref_data =ref_data.iloc[1:]
ref_data.head()

,Unnamed: 0,human_ref,chromosome,start,stop,human_ref_sequence
1,1,GRCh37/hg19,chr1,97733434,97735434,aactcctaagtccaaagtccagagtgtcatgaaagtcagatatggg...
2,2,GRCh37/hg19,chr1,194160395,194162395,ttGTCTTGCTGAATTGTGTTTATTTTAAAAATTTACACAAATAGCT...
3,3,GRCh37/hg19,chr2,171446183,171448183,aacatttagaaaatttaagtttataaggttaaaaagttacagtaag...
4,4,GRCh37/hg19,chr2,171451190,171453190,TGCTTCAAGCTAAGACTTACCCTTTGTGTGCAAATCCCAATCCCAT...
5,5,GRCh37/hg19,chr2,171451986,171453986,tactcaggctggagtgcagcggcatgaacatggctcactgcagcct...


In [30]:
for index, row in ref_data.iterrows():
  #try:

  #generate Bam file
  bam_file_path = '/content/drive/MyDrive/ACS_Research/VISDB_Data/Aligned_Sequences/'+str(row['chromosome'])+'_'+str(index)+'.bam' # Changed to .bam
  read1, read2 = create_bam_from_sequence(bam_file_path,row['human_ref_sequence'], row['human_ref_sequence'], 'hg19', row['start'])

  #generate matrix 1
  Data1, Label1=matrix_generator_uno(row['human_ref_sequence'], False)

  #save matrix 1 output
  np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/References/Experiment_1/'+str(row['chromosome'])+'_'+str(index)+'_Matrix_data.npy', Data1)
  np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/References/Experiment_1/'+str(row['chromosome'])+'_'+str(index)+'_Label_data.npy', Label1)

  #generate matrix 2
  Data2,Label2 =matrix_generator_dos (read1, read2, row['start'], row['stop'], isVirus=False)

  #save generated data
  np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/References/Experiment_2/'+str(row['chromosome'])+'_'+str(index)+'_Matrix_data.npy', Data2)
  np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/References/Experiment_2/'+str(row['chromosome'])+'_'+str(index)+'_Label_data.npy', Label2)

  #generate matrix 3
  Data3,Label3 =matrix_generator_tres(read1,read2, row['start'], row['stop'], [0,1], isVirus=False)

  #save matrix
  np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/References/Experiment_3/'+str(row['chromosome'])+'_'+str(index)+'_Matrix_data.npy', Data3)
  np.save('/content/drive/MyDrive/ACS_Research/VISDB_Data/Matrices/References/Experiment_3/'+str(row['chromosome'])+'_'+str(index)+'_Label_data.npy', Label3)

  #except:
   # print(row['human_ref_sequence'])
    #break